In [41]:

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors
import umap
import matplotlib.pyplot as plt

dataset = "go_emotions"
num = 96

df_full = pd.read_csv(f"../Adata/{dataset}/train_subset.csv").reset_index(drop=True)
texts_full = df_full["text"].tolist()
labels_full = df_full["label"].tolist()


In [42]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts_full) 

In [43]:
import numpy as np
import networkx as nx

# Załóżmy, że mamy już wyliczone embeddingi (embeddings) jak w poprzednim przykładzie.
N = len(embeddings)
k = 10  # liczba najbliższych sąsiadów do połączenia w grafie
B = num

# 1. Budowa k-NN grafu na podstawie kosinusowej podobieństwa
# (Dla efektywności można użyć struktury sąsiadów, tu prosty sposób dla jasności)
sim_matrix = np.dot(embeddings, embeddings.T)  # macierz podobieństw kosinusowych (embeddingi znormalizowane)
np.fill_diagonal(sim_matrix, -1)  # wyklucz siebie samego, ustawiając bardzo niską wartość
G = nx.DiGraph()
for i in range(N):
    # znajdź indeksy k największych podobieństw dla węzła i
    knn_idx = np.argpartition(sim_matrix[i], -k)[-k:]
    for j in knn_idx:
        G.add_edge(i, j, weight=sim_matrix[i, j])

# 2. Ranking PageRank
pr_scores = nx.pagerank(G, alpha=0.85, weight='weight')  # słownik: węzeł -> wartość PageRank

# 3. Ranking stopnia (in-degree w grafie skierowanym lub stopień w nieskierowanym)
degree_scores = dict(G.in_degree())  # liczba krawędzi skierowanych wchodzących
# (dla nieskierowanego można utworzyć G_undirected = G.to_undirected() i wtedy G_undirected.degree())

# 4. Kombinacja kryteriów - przykład: sortuj według PageRank z warunkiem różnorodności
# Najpierw lista węzłów posortowanych malejąco wg PageRank
nodes_by_pr = sorted(pr_scores.keys(), key=lambda i: pr_scores[i], reverse=True)
selected = []
selected_set = set()
for node in nodes_by_pr:
    # prosty warunek różnorodności: odrzucamy węzeł jeśli jest zbyt podobny do już wybranych
    if any(sim_matrix[node, sel] > 0.9 for sel in selected_set):
        continue
    selected.append(node)
    selected_set.add(node)
    if len(selected) >= B:
        break

# 'selected' zawiera indeksy wybranych próbek o wysokiej centralności, przefiltrowanych różnorodnością.


In [44]:
selected

[1759,
 60,
 1907,
 1798,
 939,
 683,
 429,
 852,
 184,
 774,
 390,
 1563,
 1845,
 769,
 359,
 901,
 135,
 972,
 500,
 633,
 1632,
 534,
 1522,
 1426,
 597,
 1454,
 1380,
 173,
 1148,
 493,
 1532,
 1686,
 1876,
 82,
 1815,
 1468,
 312,
 1498,
 538,
 1411,
 591,
 1523,
 568,
 1499,
 1144,
 1709,
 1909,
 1315,
 667,
 1862,
 156,
 1369,
 962,
 522,
 1896,
 716,
 618,
 883,
 575,
 1908,
 1595,
 1485,
 477,
 220,
 200,
 118,
 1272,
 1133,
 932,
 417,
 647,
 925,
 590,
 1297,
 1497,
 456,
 564,
 473,
 1116,
 269,
 485,
 1250,
 190,
 787,
 462,
 480,
 243,
 560,
 1879,
 1129,
 839,
 729,
 556,
 1665,
 664,
 1339]

In [45]:
selected_df = df_full.iloc[selected][['text', 'label']].copy()

# Zapisz do pliku CSV
selected_df.to_csv(f"selected_samples/{dataset}/page_rank.csv", index=False)
